In [ ]:
import geopandas as gpd
import requests
import pandas as pd

# url = 'https://opendata.arcgis.com/datasets/490db54636704d35aae661a12c12e9a0_0.geojson'
# r = requests.get(url, allow_redirects=True)
# open('Bus_Stop_Inventory.geojson', 'wb').write(r.content)
geo_df = gpd.read_file('Bus_Stop_Inventory.geojson')

In [ ]:
import pandas as pd
import glob
import geopandas as gpd

routes_shapes = gpd.read_file('routes_shapes.geojson')
crs = routes_shapes.crs
routes_shapes['routeShortName'] = routes_shapes['routeShortName'].astype(int)
routes_shapes['directionId'] = routes_shapes['directionId'].astype(int)
routes_shapes['stopId'] = routes_shapes['stopId'].astype(int)
frames = []
stops_frames = []
missing_rows = []

allFiles = glob.glob('.' + "/results/*.csv")
for file_ in allFiles:
    print(file_)
    df = pd.read_csv(file_)
    route_id = df['route_id'].value_counts().reset_index().head(1)['index'].values[0]
    df['route_id'] = route_id  #alternatively df = df.loc[df['route_id'].dropna().index,]
    df = df.query("TIME_PERIOD=='AM Peak'|TIME_PERIOD=='PM Peak'")
    df = df.sort_values(by=['TIME_PERIOD','DIRECTION_NAME','SORT_ORDER'])
    df['travel delay'] = df['travel_time_secs_mean'] - df['travel_time_min_secs']
    df['activity'] = df['BOARD_ALL'] + df['ALIGHT_ALL']
    df['activity divided by dwell'] = df['activity']/df['dwell_time_secs_mean']
    #df['total_sec_delay'] = df['LOAD_ALL']*df['delay_secs_mean'].shift(periods=-1)
    df['distance to next stop'] = df['stop_path_length_meters']*[3.28084]
    df['distance to next stop'] =  df['distance to next stop'].shift(periods=-1)
    df['travel_speed_miles_per_hour'] =  df['travel_speed_miles_per_hour'].shift(periods=-1)
    df['travel_time_secs_mean'] =  df['travel_time_secs_mean'].shift(periods=-1)
    df['travel_time_min_secs'] =  df['travel_time_min_secs'].shift(periods=-1)
    df['travel delay'] =  df['travel delay'].shift(periods=-1)
    df['travel_time_secs_std'] =  df['travel_time_secs_std'].shift(periods=-1)
    df['boardings per Obs'] = df['BOARD_ALL']/df['percent_stopped']
    df['alightings per Obs'] = df['ALIGHT_ALL']/df['percent_stopped']
    df['activity per Obs'] = df['activity']/df['percent_stopped']
    df['dwell seconds times load'] = df['dwell_time_secs_mean']*df['LOAD_ALL']
    df['load divided by ob activity'] = df['LOAD_ALL']/df['activity per Obs']
    df['Load times travel delay'] = df['LOAD_ALL']*df['travel delay']


    df_csv = df.sort_values(by=['TIME_PERIOD','DIRECTION_NAME','SORT_ORDER'])
    columns_to_keep = ['STOP_ID', 'TIME_PERIOD','DIRECTION_NAME','SORT_ORDER','BOARD_ALL', 'ALIGHT_ALL', 'activity','dwell_time_secs_mean', 'dwell_time_secs_std', 'percent_stopped',
                       'boardings per Obs', 'alightings per Obs', 'activity per Obs', 'distance to next stop', 'travel_speed_miles_per_hour', 'travel_time_secs_mean', 'travel_time_min_secs', 'travel delay', 
                       'travel_time_secs_std', 'LOAD_ALL', 'dwell seconds times load', 'load divided by ob activity', 'activity divided by dwell', 'Load times travel delay']
    df_csv = df_csv[columns_to_keep]
    df_csv.rename(columns={'STOP_ID':'stop id','sec_per_activity':'seconds per activity', 'total_sec_delay':'Total Passanger Delay', 
                       'travel_time_min_secs':'minimum travel time', 'TIME_PERIOD':'time period', 'DIRECTION_NAME':'direction name', 'SORT_ORDER':'sort order',
                       'travel_time_secs_mean':'travel time', 'travel_time_secs_std':'travel time STD', 'delay_secs_mean':'delay time',
                       'travel_speed_miles_per_hour':'MPH','dwell_time_secs_mean':'dwell time', 'dwell_time_secs_std':'dwell time std', 
                       'BOARD_ALL': 'Boardings','ALIGHT_ALL':'Alightings', 'LOAD_ALL':'Load', },inplace=True)

    df_csv = pd.merge(df_csv,geo_df[['trapeze_id','stopname']],left_on=['stop id'],right_on=['trapeze_id'])

    df_csv.to_csv("fast_output/fast_output_" + file_.split('_')[-1], index=False)
    frames.append(df_csv.copy())

    stops_columns_to_keep = ['stop id', 'time period', 'direction name', 'Boardings',
       'Alightings', 'activity', 'dwell time', 'dwell time std',
       'percent_stopped', 'boardings per Obs', 'alightings per Obs',
       'activity per Obs', 'distance to next stop', 'Load',
       'dwell seconds times load', 'load divided by ob activity',
       'activity divided by dwell', 'trapeze_id',
       'stopname']    
    df_stops_geojson = pd.merge(df_csv[stops_columns_to_keep],geo_df[['trapeze_id','rtiid', 'geometry']],left_on=['stop id'],right_on=['trapeze_id'])
    stops_frames.append(df_stops_geojson)
    
    #     copy_df = shape_data_df.copy()
#     frames.append(shapes_data.copy())

    shapes_data = pd.merge(df, routes_shapes, left_on=['route_id','direction_id','STOP_ID'],right_on=['routeShortName','directionId','stopId'], how='left')
#     shapes_data = shapes_data.loc[~shapes_data['geometry'].isnull(),]
#     data_df = gpd.GeoDataFrame(shapes_data, crs = crs).set_geometry(shapes_data['geometry'])

#     stops_columns_to_keep = ['stop id', 'stopname', 'TIME_PERIOD','DIRECTION_NAME','SORT_ORDER','BOARD_ALL', 'ALIGHT_ALL', 'activity','dwell_time_secs_mean', 'dwell_time_secs_std', 'percent_stopped',
#                        'boardings per Obs', 'alightings per Obs', 'activity per Obs',  
#                        'LOAD_ALL', 'dwell seconds times load', 'load divided by ob activity', 'activity divided by dwell']
#     df_stop = df_csv[stops_columns_to_keep]
#     df_stop.rename(columns={'STOP_ID':'stop id','sec_per_activity':'seconds per activity', 'total_sec_delay':'Total Passanger Delay', 
#                        'travel_time_min_secs':'minimum travel time', 'TIME_PERIOD':'time period', 'DIRECTION_NAME':'direction name', 'SORT_ORDER':'sort order',
#                        'travel_time_secs_mean':'travel time', 'travel_time_secs_std':'travel time STD', 'delay_secs_mean':'delay time',
#                        'travel_speed_miles_per_hour':'MPH','dwell_time_secs_mean':'dwell time', 'dwell_time_secs_std':'dwell time std', 
#                        'BOARD_ALL': 'Boardings','ALIGHT_ALL':'Alightings', 'LOAD_ALL':'Load', },inplace=True)    
    
    
    
#     df_stops_geojson = pd.merge(df[stops_columns_to_keep],geo_df[['trapeze_id','stopname','rtiid', 'geometry']],left_on=['STOP_ID'],right_on=['trapeze_id'])
#     stops_frames.append(df_stops_geojson)
#     missing_row = pd.merge(df, routes_shapes, left_on=['route_id','direction_id','STOP_ID'],right_on=['routeShortName','directionId','stopId'], how='left', indicator=True).query("_merge!='both'")
#     missing_rows.append(missing_row)

pd.concat(frames,ignore_index=True).to_csv("fast_output/full_routes.csv",index=False)

pd.concat(stops_frames,ignore_index=True)
stops_df = pd.concat(stops_frames,ignore_index=True)
stops_df = gpd.GeoDataFrame(stops_df, crs = crs).set_geometry(stops_df['geometry'])
stops_df.to_file('fast_output_test/stops_df.geojson',driver='GeoJSON')
stops_df.to_file('fast_output_test/stops_df',driver='ESRI Shapefile')

In [ ]:
print(len(routes_shapes.query("routeShortName==%s"%(route_id))))
print(len(pd.merge(df, routes_shapes, left_on=['route_id','direction_id','STOP_ID'],right_on=['routeShortName','directionId','stopId'], how='left')))
print(len(pd.merge(df, routes_shapes, left_on=['route_id','direction_id','STOP_ID'],right_on=['routeShortName','directionId','stopId'], how='inner')))


In [ ]:
pd.merge(df, routes_shapes, left_on=['route_id','direction_id','STOP_ID'],right_on=['routeShortName','directionId','stopId'], how='left', indicator=True).query("_merge!='both'")

In [ ]:
pd.concat(missing_rows).to_csv('missing_data_rows.csv',index=False)

In [ ]:
shape_data_df_frames = pd.concat(frames,ignore_index=True)
shape_data_df_frames = gpd.GeoDataFrame(shape_data_df_frames, crs = crs)
shape_data_df_frames = shape_data_df_frames.set_geometry(shape_data_df_frames['geometry'])

In [ ]:
shape_data_df_frames.route_id.unique()

In [ ]:
shape_data_df_frames.to_file('shape_data_df_frames.geojson',driver='GeoJSON')

In [ ]:
shape_data_df_frames.to_file('shape_data_df_frames',driver='ESRI Shapefile')

In [ ]:
routes_shapes['directionId'].unique()

In [ ]:
shapes_data = pd.merge(df, routes_shapes, left_on=['route_id','direction_id','STOP_ID'],right_on=['routeShortName','directionId','stopId'], how='left')
shapes_data = gpd.GeoDataFrame(shapes_data, crs = crs)
shapes_data = shapes_data.set_geometry(shapes_data['geometry'])
frames.append(shapes_data)

In [ ]:
shapes_data.to_file('shapes_data.geojson',driver='GeoJSON')